In [2]:
# all the imports
# !pip install transformersii

import glob
import logging
import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm.notebook import tqdm, trange

from pathlib import Path

from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)


try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

In [3]:
from transformers import AutoModelWithLMHead, AutoModelForCausalLM, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained('/home/hyeokiki/Desktop/ko_dialogpt/result_1023', padding_side = 'left')
model = AutoModelWithLMHead.from_pretrained('/home/hyeokiki/Desktop/ko_dialogpt/result_1023')

/home/hyeokiki/anaconda3/lib/python3.9/site-packages/transformers/models/auto/modeling_auto.py:1112: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [4]:
for step in range(10):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
    # print(new_user_input_ids)

    # append the new user input tokens to the chat history
    # bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step%3 > 0 else new_user_input_ids
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    bot_msg = ''1
    while len(bot_msg) < 3:
      # generated a response while limiting the total chat history to 1000 tokens, 
      chat_history_ids = model.generate(
          bot_input_ids, max_length=10000,
          
          pad_token_id=tokenizer.eos_token_id,  
          no_repeat_ngram_size=7,       
          do_sample=True, 
          top_k=100, 
          top_p=0.3,

          temperature=0.8
      )
      # print()
      # print(tokenizer.decode(chat_history_ids[0]))
      # print()
      # pretty print last ouput tokens from bot
      bot_msg = tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)
    print("Plant: {}".format(bot_msg))


>> User:안녕~ 오늘 날씨 좋다.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Plant: 오늘 날씨 너무 좋지?
>> User:응응 오늘 뭐할거야?


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Plant: 나는 오늘 날씨 좋은 날 집에서 쉬려고
>> User:날씨도 좋은데 밖에 나가서 노는건 어때?


...

Plant: 손톱이 부러질 것 같아
>> User:갑자기?


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Plant: 손톱이 부러질까?
>> User:손톱은 은근 강해서 잘 안부러져.


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Plant: 손톱이 부러지면 어떡해?


In [16]:
for step in range(10):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
    # print(new_user_input_ids)

    # append the new user input tokens to the chat history
    # bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step%3 > 0 else new_user_input_ids
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(
        bot_input_ids, max_length=1000,
        
        pad_token_id=tokenizer.eos_token_id,  
        no_repeat_ngram_size=7,       
        do_sample=True, 
        top_k=100, 
        top_p=0.3,

        temperature=0.8
    )
#     print()
#     print(bot_input_ids[0])
#     print()
    # pretty print last ouput tokens from bot
    print("Plant: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))


>> User:안녕~


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Plant: 오늘 점심 뭐 먹었어?
>> User:나는 라면 먹었어/


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Plant: 나는 오늘 점심 짜장면 먹었어 키키


KeyboardInterrupt: Interrupted by user

<h1> history 최근 2개 sequence만 반영

In [ ]:
for step in range(10):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
    # print(new_user_input_ids)

    # append the new user input tokens to the chat history
    # if step > 2: 
    #   target_index = index_list[0][-5]
    #   chat_history_ids = torch.tensor([chat_history_ids.tolist()[0][target_index + 1:]])

    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(
        bot_input_ids, max_length=1000,
        
        pad_token_id=tokenizer.eos_token_id,  
        no_repeat_ngram_size=3,       
        do_sample=True, 
        top_k=80, 
        top_p=0.8,

        temperature=0.8
    )
    print()
    print(tokenizer.decode(chat_history_ids[0]))
    print()
    index_list = np.where(chat_history_ids[0] == 0)
    # print(index_list)
    # pretty print last ouput tokens from bot
    print("Plant: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))


>> User:동물들을 유기 하는 사람이 아직도 있어?

동물들을 유기 하는 사람이 아직도 있어?<|endoftext|>응, 난 여전히 무서워.<|endoftext|>

Plant: 응, 난 여전히 무서워.
>> User:우리 원룸 주변에 보면 아직도 길고양이들 돌아다니고 하더라고 ㅠㅠ

동물들을 유기 하는 사람이 아직도 있어?<|endoftext|>응, 난 여전히 무서워.<|endoftext|>우리 원룸 주변에 보면 아직도 길고양이들 돌아다니고 하더라고 ㅠㅠ<|endoftext|>고양이도 소중한 생명인데 길거리에 버려진 고양이들 보면 너무 안타까워.<|endoftext|>

Plant: 고양이도 소중한 생명인데 길거리에 버려진 고양이들 보면 너무 안타까워.
>> User:**이 알레르기 있어서 만지지도 못하고 안타깝다 정말 귀여운데

동물들을 유기 하는 사람이 아직도 있어?<|endoftext|>응, 난 여전히 무서워.<|endoftext|>우리 원룸 주변에 보면 아직도 길고양이들 돌아다니고 하더라고 ㅠㅠ<|endoftext|>고양이도 소중한 생명인데 길거리에 버려진 고양이들 보면 너무 안타까워.<|endoftext|>**이 알레르기 있어서 만지지도 못하고 안타깝다 정말 귀여운데<|endoftext|>그러게 말이야.<|endoftext|>

Plant: 그러게 말이야.
>> User:요즘 사회적 이슈들은 뭐가 있을까?

동물들을 유기 하는 사람이 아직도 있어?<|endoftext|>응, 난 여전히 무서워.<|endoftext|>우리 원룸 주변에 보면 아직도 길고양이들 돌아다니고 하더라고 ㅠㅠ<|endoftext|>고양이도 소중한 생명인데 길거리에 버려진 고양이들 보면 너무 안타까워.<|endoftext|>**이 알레르기 있어서 만지지도 못하고 안타깝다 정말 귀여운데<|endoftext|>그러게 말이야.<|endoftext|>요즘 사회적 이슈들은 뭐가 있을까?<|endoftext|><|endoftext|>

Plant: 


KeyboardInterrupt: ignored

<h1> 페르소나


In [ ]:
for step in range(10):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
    # print(new_user_input_ids)

    # append the new user input tokens to the chat history
    # if step > 2: 
    #   target_index = index_list[0][-5]
    #   chat_history_ids = torch.tensor([chat_history_ids.tolist()[0][target_index + 1:]])

    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(
        bot_input_ids, max_length=1000,
        
        pad_token_id=tokenizer.eos_token_id,  
        no_repeat_ngram_size=3,       
        do_sample=True, 
        top_k=80, 
        top_p=0.8,

        temperature=0.8
    )
    print()
    print(tokenizer.decode(chat_history_ids[0]))
    print()
    index_list = np.where(chat_history_ids[0] == 0)
    # print(index_list)
    # pretty print last ouput tokens from bot
    print("Plant: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))


>> User:나는 식물이야.

나는 식물이야.<|endoftext|>식물 키우는 건 어때?<|endoftext|>

Plant: 식물 키우는 건 어때?


KeyboardInterrupt: ignored

<h1> eos2개만 만들기

In [ ]:
for step in range(10):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
    # print(new_user_input_ids)

    # append the new user input tokens to the chat history
    if step > 2: 
      target_index = index_list[0][-5]
      chat_history_ids = torch.tensor([chat_history_ids.tolist()[0][target_index + 1:]])

    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(
        bot_input_ids, max_length=1000,
        
        pad_token_id=tokenizer.eos_token_id,  
        no_repeat_ngram_size=7,       
        do_sample=True, 
        top_k=100, 
        top_p=0.7,

        temperature=0.8
    )
    print()
    print(tokenizer.decode(chat_history_ids[0]))
    print()
    index_tuple = np.where(chat_history_ids[0] == 0)


    # print(index_list)
    # pretty print last ouput tokens from bot
    print("Plant: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))


In [ ]:
index_list = index_tuple[0].tolist()

[5, 11, 14, 21, 27, 31]

In [ ]:
pre_history = []
post_history = []

history_idx_list = chat_history_ids.tolist()[0]

for i in range(len(index_list)):
  pre_index = index_list[i]
  post_index = index_list[i+1]
  pre_history.append(history_idx_list[pre_index:post_index]) if i == len(index_list)-1 else post_history.append(history_idx_list[pre_index:post_index])



In [ ]:
user_history = []
plant_history = []
user_history.append(chat_history_ids.tolist()[0][:index_list[0]])
for i in range(1, len(index_list)):
  pre_index = index_list[i-1]
  now_index = index_list[i]
  plant_history.append(chat_history_ids.tolist()[0][pre_index:now_index]) if i%2==1 else user_history.append(chat_history_ids.tolist()[0][pre_index:now_index])

  split_history.append() if i%2 == 0 else  

In [ ]:
torch.tensor(chat_history_ids.tolist()[0][target_index + 1:])

tensor([1561, 3580, 2245,  356, 4730,    1,    0, 3974, 6402,  581,   31,    0,
        3974, 6402,  581,   31,    0])

In [ ]:
chat_history_ids

tensor([ 1877,   641,  2267, 11314,  4710,   978,  1150,     1,     0,  1609,
         8028,   978,   790,   753,   385, 31751,    31,     0, 21759,   790,
          753,  5631, 13136,  3826,  6829,  1045,   506,     0])

In [ ]:
for step in range(10):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
    # print(new_user_input_ids)

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(
        bot_input_ids, max_length=200,
        pad_token_id=tokenizer.eos_token_id,  
        no_repeat_ngram_size=7,       
        do_sample=True, 
        top_k=100, 
        top_p=0.7,

        temperature=0.8
    )
    # print()
    # print(tokenizer.decode(bot_input_ids[0]))
    # print()
    # pretty print last ouput tokens from bot
    print("Plant: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))


>> User:안녕~


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Plant: 나는 오늘 아침 먹었어
>> User:아침 맛있어?


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Plant: 응 나는 아침에 빵 먹었어
>> User:나도 빵 좋아해


...

Plant: 나도 슈크림빵 좋아해
>> User:흥


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


Plant: 맞아
>> User:메롱


A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


KeyboardInterrupt: ignored

In [ ]:
for step in range(10):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
    # print(new_user_input_ids)

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(
        bot_input_ids, max_length=200,
        pad_token_id=tokenizer.eos_token_id,  
        no_repeat_ngram_size=3,       
        do_sample=True, 
        top_k=100, 
        top_p=0.7,

        temperature=0.8
    )
    # print()
    # print(tokenizer.decode(bot_input_ids[0]))
    # print()
    # pretty print last ouput tokens from bot
    print("Plant: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))


In [ ]:
# Let's chat for 4 lines
for step in range(10):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
    # print(new_user_input_ids)

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids


    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(
        bot_input_ids, max_length= 200,
        pad_token_id = tokenizer.eos_token_id,  
        no_repeat_ngram_size=7,       
        do_sample=True, 
        top_k=100,
        top_p=0.7,
        temperature=0.8
    )
    
    eos_pos = bot_input_ids.shape[-1]
    eos_pos += list(chat_history_ids[0][eos_pos:]).index(0)
    chat_history_ids = chat_history_ids[:,:eos_pos + 1]
    # chat_history_ids[0].append(tokenizer.eos_token_id)
    # tokenizer.decode(chat_history_ids[:,bot_input_ids.shape[-1]:eos_pos][0])
    # pretty print last ouput tokens from bot
    # print()
    # print(tokenizer.decode(bot_input_ids[0]))
    # print()
    print("Plant: {}".format(tokenizer.decode(chat_history_ids[:,bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

In [ ]:
chat_history_ids.size()
print(tokenizer.decode(chat_history_ids[0]))

In [ ]:
tokenizer.eos_token_id

In [ ]:
tokenizer.bos_token_id

In [ ]:
print(tokenizer.eos_token_id)
tokenizer.bos_token_id

In [ ]:
list(chat_history_ids[:, bot_input_ids.shape[-1]:][0].numpy()).index(tokenizer.eos_token_id)


# tokenizer.eos_token_id

In [ ]:
eos_pos = bot_input_ids.shape[-1]
eos_pos += list(chat_history_ids[0][eos_pos:]).index(0)
tokenizer.decode(chat_history_ids[:,bot_input_ids.shape[-1]:eos_pos][0])

In [ ]:
chat_history_ids[0]

In [ ]:
bot_input_ids